In [ ]:
if not os.path.exists('code'):
    os.makedirs('code')

In [1]:
import src.company_deduplcation as cmp_dup
df = cmp_dup.process_company_duplicator("./data/Company_Data.csv")


2024-Sep-16 11:07:24 - company_deduplication - INFO - Started processing...
2024-Sep-16 11:07:24 - company_deduplication - INFO - reading input file and preparing dataframe.
2024-Sep-16 11:07:24 - company_deduplication - INFO - Finding duplicate rows.
2024-Sep-16 11:07:24 - company_deduplication - INFO - Assigning group id to duplicate rows.
2024-Sep-16 11:07:24 - company_deduplication - INFO - duplicate identification Total Processing Time : 0.12 Seconds
2024-Sep-16 11:07:24 - company_deduplication - INFO - Started duplicate resolution.
2024-Sep-16 11:07:24 - company_deduplication - INFO - Saving file with duplicate groups and Keep delete flag ./output//Company_Data_with_flag.csv
2024-Sep-16 11:07:24 - company_deduplication - INFO - duplicate resolution Total Processing Time : 0.03 Seconds


In [1]:
from src.settings import constants as const
from src.util import helper 
import pandas as pd

def read_file_prep_dataframe(file_path, file_ext):
	if file_ext.lower() == ".csv":
		df = pd.read_csv(file_path,encoding='cp1252')
	else:
		df = pd.read_excel(file_path) 

	df = df.fillna("") # Fill Empty value as all columns are strings only
		#df.to_excel("./output/crm_contact_with_row_no.xlsx")

	# Clean text of all dataframe text data one time
	df = df.map(lambda x: helper.clean_contact_data(x) if isinstance(x, str) else x)

	## Add New Columns to Add into original Dataframe
	df.insert(0,const.COL_DUP_GROUP_ID,999999)
	if const.COL_ID not in df.columns:
		# Add RowNum to DataFrame
		df[const.COL_ID] = range(0+1,len(df)+1)
	df.insert(2,const.COL_ERROR_RATE,100.00)
	df[const.COL_FUZZ_SIMILARITY] = 0.0
	df[const.COL_LEVENSHTEIN_SIMILARITY] = 0.0
	df[const.COL_JARO_SIMILARITY] = 0.0
	df[const.COL_DUP_ROW_GROUP] = ""
	return df

In [ ]:
import re

def clean_company_name(company_name):
    # Define keywords to be removed
    keywords = ['LLP', 'LLC', 'INC', 'PVT', 'LTD', 'CORP', 'CO']
    # Create a regex pattern to match any of the keywords
    pattern = r'\b(?:' + '|'.join(keywords) + r')\b\.?'
    # Substitute the keywords with an empty string
    cleaned_name = re.sub(pattern, '', company_name, flags=re.IGNORECASE)
    # Remove extra spaces
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()
    return cleaned_name

# Example usage
company_names = [
    "Tech Innovations LLC",
    "Green Earth Inc.",
    "Alpha Beta Corp",
    "Creative Minds LLP",
    "XYZ Pvt Ltd"
]

cleaned_names = [clean_company_name(name) for name in company_names]
print(cleaned_names)


In [ ]:
def remove_https(url):
    # Check if the URL starts with "https://"
    if url.startswith("https://"):
        # Remove "https://" from the URL
        return url[8:]
    return url

# Example usage
websites = [
    "https://www.example.com",
    "https://genexa.ai",
    "http://www.test.com",
    "www.nodomain.com"
]
cleaned_websites = [remove_https(website) for website in websites]
print(cleaned_websites)


In [36]:
import pandas as pd
df = pd.read_csv("./data/Company_Data.csv")

In [ ]:
df.columns

In [ ]:
df["Company Name"].apply(lambda x: clean_company_name(x))

In [9]:
from rapidfuzz import fuzz
import jellyfish as jfish
import Levenshtein
def compare_rows(src_row,trg_row):
	"""
		Based On Compare Columns, Calculate Levenstine, Jaro Winkle and Fuzzy raito and select
		where Error rate lowest or similarity ratio is higher
	"""
	comparesion_list = []
	id = 0
	min_error = 100.0
	min_error_id = -1
	COMPARE_COLUMNS=[{ "name" : "Rule1", "src_cols":["Link",], "trg_cols" : ["Link",]},]
	for comp_columns in COMPARE_COLUMNS:
		str_src = get_col_value(src_row,comp_columns['src_cols'])
		str_trg = get_col_value(trg_row,comp_columns['trg_cols'])
		res = {}
		res["name"] = comp_columns["name"]
		res["f" + str(id)] = get_fuzzy_similarity(str_src,str_trg) 
		res["l" + str(id)] = get_levenshtein_similarity(str_src,str_trg)
		res["j" + str(id)] = get_jaro_winkler_similarity(str_src,str_trg)
		#logger.info(f"src val : {str_src}, trg val : {str_trg}, fuzz : {res['f' + str(id)] }, levst : {res['l' + str(id)]}, jero : {res['j' + str(id)]}")
		res["e" + str(id)] =  (1.0-(float(res["f" + str(id)]) + float(res["l" + str(id)]) + float(res["j" + str(id)]) )/3)*100
		if ( float(res["e" + str(id)]) < min_error):
			min_error = res["e" + str(id)]
			min_error_id = id
		comparesion_list.append(res)
		id += 1	
	#logger.info(f"min error rate : {min_error}, min error id : {min_error_id}")	
	return min_error_id, comparesion_list[min_error_id]

def get_fuzzy_similarity(str_src,str_trg):
	if  (len(str_src) != 0 and len(str_trg) != 0):
		return (fuzz.token_set_ratio(str_src, str_trg)/100)
	else:
		return 0.0	

def get_levenshtein_similarity(str_src,str_trg):
	"""
	Levenshtein distance measures the minimum number of single-character edits required to change one string into another.
	"""
	if  (len(str_src) != 0 and len(str_trg) != 0):
		return Levenshtein.ratio(str_src, str_trg)
	else:
		return 0.0	

def get_jaro_winkler_similarity(str_src,str_trg):
	if  (len(str_src) != 0 and len(str_trg) != 0):
		return jfish.jaro_winkler_similarity(str_src, str_trg)
	else:
		return 0.0	

def get_col_value(row : any,col_list : list) -> str:
	val_list = []
	for col in col_list:
		val_list.append(str(row[col]).strip())
	return " ".join(val_list)

import src.settings.constants as const
def find_dup_row_by_sequence(df,sort_col):
	df = df.sort_values(by=[sort_col])
	list_dup_rows = []
	# Now Iterate through Each record and try to find out Group
	for src_idx in range(len(df)):
		src_row = df.iloc[src_idx]
		for trg_idx in range(len(df)):
			trg_row = df.iloc[trg_idx]
			#logger.info(f"source index {src_idx} ,target index : {trg_idx}")
			if trg_idx > src_idx:
				dup_grp = {}
				dup_grp[const.COL_SOURCE_ID] = src_row[const.COL_ID]
				dup_grp[const.COL_TARGET_ID] = trg_row[const.COL_ID]
				dup_grp["src_Link"] = str(src_row["Link"])
				dup_grp["trg_Link"] = str(trg_row["Link"])
				id, rsp = compare_rows(src_row,trg_row)
				#logger.info(id, rsp)
				dup_grp[const.COL_FUZZ_SIMILARITY] = float(rsp["f" + str(id)])
				dup_grp[const.COL_LEVENSHTEIN_SIMILARITY] = float(rsp["l" + str(id)])
				dup_grp[const.COL_JARO_SIMILARITY] = float(rsp["j" + str(id)])
				dup_grp[const.COL_ERROR_RATE] = float(rsp["e" + str(id)])
				if dup_grp[const.COL_ERROR_RATE] <= const.MAX_ERROR_RATE:
					list_dup_rows.append(dup_grp)
	return list_dup_rows

In [10]:
def find_dup_row_by_company_website(df,col_first_name="Company Name"):
	list_dup_rows = []
	df_dup = df.groupby(["Link"], as_index=False).agg(count=("Link", 'count'))
	df_dup = df_dup[df_dup["count"] > 1]
	for id, row in df_dup.iterrows():
		df_flt = df[df["Link"] == row["Link"]]
		df_flt = df_flt.sort_values(by=[col_first_name])
		list_dup_rows.extend(find_dup_row_by_sequence(df=df_flt,sort_col=col_first_name))
	
	# # Now Iterate through dup Rows and If last name matching Error rate is higher than threshold,
	# # Remove those rows from List.
	# list_dup_rows = remove_dup_rows_by_lastname_match_similarity(list_dup_rows)
	
	df_dup_rows = pd.DataFrame.from_dict(list_dup_rows)
	return df_dup_rows

In [5]:
import src.settings.constants as const
def find_dup_row_by_sequence(df,sort_col):
	df = df.sort_values(by=[sort_col])
	list_dup_rows = []
	# Now Iterate through Each record and try to find out Group
	for src_idx in range(len(df)):
		src_row = df.iloc[src_idx]
		for trg_idx in range(len(df)):
			trg_row = df.iloc[trg_idx]
			#logger.info(f"source index {src_idx} ,target index : {trg_idx}")
			if trg_idx > src_idx:
				dup_grp = {}
				dup_grp[const.COL_SOURCE_ID] = src_row[const.COL_ID]
				dup_grp[const.COL_TARGET_ID] = trg_row[const.COL_ID]
				dup_grp["src_Link"] = str(src_row["Link"])
				dup_grp["trg_Link"] = str(trg_row["Link"])
				id, rsp = compare_rows(src_row,trg_row)
				#logger.info(id, rsp)
				dup_grp[const.COL_FUZZ_SIMILARITY] = float(rsp["f" + str(id)])
				dup_grp[const.COL_LEVENSHTEIN_SIMILARITY] = float(rsp["l" + str(id)])
				dup_grp[const.COL_JARO_SIMILARITY] = float(rsp["j" + str(id)])
				dup_grp[const.COL_ERROR_RATE] = float(rsp["e" + str(id)])
				if dup_grp[const.COL_ERROR_RATE] <= const.MAX_ERROR_RATE:
					list_dup_rows.append(dup_grp)
	return list_dup_rows


def find_dup_row_by_company_website(df,col_first_name="Company Name"):
	list_dup_rows = []
	df_dup = df.groupby(["Link"], as_index=False).agg(count=("Link", 'count'))
	df_dup = df_dup[df_dup["count"] > 1]
	for id, row in df_dup.iterrows():
		df_flt = df[df["Link"] == row["Link"]]
		df_flt = df_flt.sort_values(by=[col_first_name])
		list_dup_rows.extend(find_dup_row_by_sequence(df=df_flt,sort_col=col_first_name))
	
	# # Now Iterate through dup Rows and If last name matching Error rate is higher than threshold,
	# # Remove those rows from List.
	# list_dup_rows = remove_dup_rows_by_lastname_match_similarity(list_dup_rows)
	
	df_dup_rows = pd.DataFrame.from_dict(list_dup_rows)
	return df_dup_rows

In [31]:
import os
import time
from src.util import helper

from src.settings import constants as const
from src.util import helper 
import pandas as pd

def read_file_prep_dataframe(file_path, file_ext):
	if file_ext.lower() == ".csv":
		df = pd.read_csv(file_path,encoding='cp1252')
	else:
		df = pd.read_excel(file_path) 

	df = df.fillna("") # Fill Empty value as all columns are strings only
		#df.to_excel("./output/crm_contact_with_row_no.xlsx")

	# Clean text of all dataframe text data one time
	df = df.map(lambda x: helper.clean_contact_data(x) if isinstance(x, str) else x)

	## Add New Columns to Add into original Dataframe
	df.insert(0,const.COL_DUP_GROUP_ID,999999)
	if const.COL_ID not in df.columns:
		# Add RowNum to DataFrame
		df[const.COL_ID] = range(0+1,len(df)+1)
	df.insert(2,const.COL_ERROR_RATE,100.00)
	df[const.COL_FUZZ_SIMILARITY] = 0.0
	df[const.COL_LEVENSHTEIN_SIMILARITY] = 0.0
	df[const.COL_JARO_SIMILARITY] = 0.0
	df[const.COL_DUP_ROW_GROUP] = ""
	return df

def save_file(df: any, file_path : str, ext:str):
	if ext.lower() == ".xlsx":
		df.to_excel(file_path,index=False)
	else:
		df.to_csv(file_path,index=False)

def load_file(file_path : str, ext:str):
	if ext.lower() == ".csv":
		df = pd.read_csv(file_path,encoding='cp1252')
	else:
		df = pd.read_excel(file_path) 
	return df

def process_contact_duplicator(file_path : str):
	start_time = time.time()	
	try:
		if not os.path.exists(file_path):
			return
		name, ext = helper.get_file_name_and_extension(file_path)
		if not (ext.lower() == ".csv" or ext.lower() == ".xlsx"):
			return 
		# Step 1
		file_out = const.OUTPUT_PATH + "/" +  name + "_org"+ext
		if not os.path.exists(file_out):
			df = read_file_prep_dataframe(file_path,ext)
			save_file(df,file_out,ext)
		else:
			df = load_file(file_out,ext)
			#return df

		file_out = const.OUTPUT_PATH + "/" +  name + "_dup"+ext
		if not os.path.exists(file_out):
			#Step 2
			df_dup = find_dup_row_by_company_website(df)
			#Step 3
	
			df = assign_dup_row_groups(df,df_dup)
			df.to_csv("Output_1.csv",date_format=str)
			df.to_excel("Output_1.xlsx")
			save_file(df,file_out,ext)
			return df
	
		# #Step 4 Assing the Keep / Duplicate Flag to each row based on research.
		# df = assign_keep_duplicate_flag(df)

		# file_out = const.OUTPUT_PATH + "/" +  name + "_with_flag"+ext
		# logger.info(f"Saving file with duplicate groups and Keep delete flaf {file_out}")
		# save_file(df,file_out,ext)
	except Exception as e:
		print(e)


In [32]:
def find_group(row, row_to_group, group_id):
	if row in row_to_group:
			return row_to_group[row]
	row_to_group[row] = group_id
	return group_id
def union_groups(row1, row2, row_to_group, group_id):
	group1 = find_group(row1, row_to_group, group_id)
	group2 = find_group(row2, row_to_group, group_id)
	if group1 != group2:
			for row in row_to_group:
					if row_to_group[row] == group2:
							row_to_group[row] = group1

def assign_group_ids(pairs):
	row_to_group = {}
	group_id = 1

	for src_row, trg_row in pairs:
			if src_row not in row_to_group and trg_row not in row_to_group:
					row_to_group[src_row] = group_id
					row_to_group[trg_row] = group_id
					group_id += 1
			elif src_row in row_to_group and trg_row not in row_to_group:
					row_to_group[trg_row] = row_to_group[src_row]
			elif trg_row in row_to_group and src_row not in row_to_group:
					row_to_group[src_row] = row_to_group[trg_row]
			else:
					union_groups(src_row, trg_row, row_to_group, group_id)

	# Ensure each component has a unique group ID
	unique_groups = {}
	current_group_id = 1
	for row in row_to_group:
			old_group_id = row_to_group[row]
			if old_group_id not in unique_groups:
					unique_groups[old_group_id] = current_group_id
					current_group_id += 1
			row_to_group[row] = unique_groups[old_group_id]

	return row_to_group

def assign_dup_row_groups(df,dup_df):
	dup_df = dup_df.sort_values(by=["src_row_no"])		

	# This will give Group ID assign to each row	
	group_ids = assign_group_ids(list(dup_df[['src_row_no', 'trg_row_no']].itertuples(index=False,name=None)))
	
	# In this Loop If any two record qualify or three records, One Record data is missing.
	for src_idx in range(len(dup_df)):
		row = dup_df.iloc[src_idx]
		
		if df[df[const.COL_ID] == row[const.COL_SOURCE_ID]][const.COL_DUP_GROUP_ID].values[0] == 999999:
			df.loc[df[const.COL_ID] == row[const.COL_SOURCE_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],group_ids[row[const.COL_SOURCE_ID]],str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]
		else:
			df.loc[df[const.COL_ID] == row[const.COL_TARGET_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],group_ids[row[const.COL_TARGET_ID]],str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]
	
	#Loop Throuh again with Dup Record for any Source Or Target Row is missing For Group ID Update
	for src_idx in range(len(dup_df)):
		row = dup_df.iloc[src_idx]
		grp_id = df[df[const.COL_ID] == row[const.COL_SOURCE_ID]][const.COL_DUP_GROUP_ID].values[0]
		if grp_id != 999999:
			df.loc[df[const.COL_ID] == row[const.COL_TARGET_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],grp_id,str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]
		else:
			grp_id = df[df[const.COL_ID] == row[const.COL_TARGET_ID]][const.COL_DUP_GROUP_ID].values[0]
			df.loc[df[const.COL_ID] == row[const.COL_SOURCE_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],grp_id,str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]

	return df

In [ ]:
value = process_contact_duplicator("./data/Company_Data.csv")

In [20]:
value['row_group'] =value['row_group'].astype(str)

In [ ]:
value.head()

In [23]:
value.to_csv("output_2.csv",index=False,quoting=1)

In [ ]:
list_dup_rows = []
df_dup = df.groupby(["Link"], as_index=False).agg(count=("Link", 'count'))
df_dup = df_dup[df_dup["count"] > 1]
for id, row in df_dup.iterrows():
	df_flt = df[df["Link"] == row["Link"]]
	print(df_flt)
	df_flt = df_flt.sort_values(by=["Company Name"])

In [ ]:
value =find_dup_row_by_sequence(df=df_flt,sort_col="Company Name")

In [ ]:
value.head()

In [13]:
duplicat = pd.DataFrame(value)
duplicat.to_csv("OutputFiles.csv")


In [ ]:
for src_idx in range(len(df_flt)):
		src_row = df.iloc[src_idx]
print(src_idx)
src_row["Link"]

In [32]:
import src.settings.constants as const
def find_dup_row_by_sequence(df,sort_col):
	df = df.sort_values(by=[sort_col])
	list_dup_rows = []
	# Now Iterate through Each record and try to find out Group
	for src_idx in range(len(df)):
		src_row = df.iloc[src_idx]
		for trg_idx in range(len(df)):
			trg_row = df.iloc[trg_idx]
			#logger.info(f"source index {src_idx} ,target index : {trg_idx}")
			if trg_idx > src_idx:
				dup_grp = {}
				dup_grp[const.COL_SOURCE_ID] = src_row[const.COL_ID]
				dup_grp[const.COL_TARGET_ID] = trg_row[const.COL_ID]
				dup_grp["src_Link"] = str(src_row["Link"])
				dup_grp["trg_Link"] = str(trg_row["Link"])
				id, rsp = compare_rows(src_row,trg_row)
				#logger.info(id, rsp)
				dup_grp[const.COL_FUZZ_SIMILARITY] = float(rsp["f" + str(id)])
				dup_grp[const.COL_LEVENSHTEIN_SIMILARITY] = float(rsp["l" + str(id)])
				dup_grp[const.COL_JARO_SIMILARITY] = float(rsp["j" + str(id)])
				dup_grp[const.COL_ERROR_RATE] = float(rsp["e" + str(id)])
				if dup_grp[const.COL_ERROR_RATE] <= const.MAX_ERROR_RATE:
					list_dup_rows.append(dup_grp)
	return list_dup_rows

In [1]:
import pandas as pd
df = pd.read_csv("./data/Company_Data_1.1.csv")

### If website does not exist, Need to fetch website first and then Deduplication.

In [34]:
from rapidfuzz import fuzz
import jellyfish as jfish
import Levenshtein
def compare_rows(src_row,trg_row):
	"""
		Based On Compare Columns, Calculate Levenstine, Jaro Winkle and Fuzzy raito and select
		where Error rate lowest or similarity ratio is higher
	"""
	comparesion_list = []
	id = 0
	min_error = 100.0
	min_error_id = -1
	COMPARE_COLUMNS=[{ "name" : "Rule1", "src_cols":["Clean_Company",], "trg_cols" : ["Clean_Company",]},
				  {"name" : "Rule2", "src_cols":["Clean_Website"],"trg_cols" : ["Clean_Website"]}]
	for comp_columns in COMPARE_COLUMNS:
		str_src = get_col_value(src_row,comp_columns['src_cols'])
		str_trg = get_col_value(trg_row,comp_columns['trg_cols'])
		res = {}
		res["name"] = comp_columns["name"]
		res["f" + str(id)] = get_fuzzy_similarity(str_src,str_trg) 
		res["l" + str(id)] = get_levenshtein_similarity(str_src,str_trg)
		res["j" + str(id)] = get_jaro_winkler_similarity(str_src,str_trg)
		#logger.info(f"src val : {str_src}, trg val : {str_trg}, fuzz : {res['f' + str(id)] }, levst : {res['l' + str(id)]}, jero : {res['j' + str(id)]}")
		res["e" + str(id)] =  (1.0-(float(res["f" + str(id)]) + float(res["l" + str(id)]) + float(res["j" + str(id)]) )/3)*100
		if ( float(res["e" + str(id)]) < min_error):
			min_error = res["e" + str(id)]
			min_error_id = id
		comparesion_list.append(res)
		id += 1	
	#logger.info(f"min error rate : {min_error}, min error id : {min_error_id}")	
	return min_error_id, comparesion_list[min_error_id]

def get_fuzzy_similarity(str_src,str_trg):
	if  (len(str_src) != 0 and len(str_trg) != 0):
		return (fuzz.token_set_ratio(str_src, str_trg)/100)
	else:
		return 0.0	

def get_levenshtein_similarity(str_src,str_trg):
	"""
	Levenshtein distance measures the minimum number of single-character edits required to change one string into another.
	"""
	if  (len(str_src) != 0 and len(str_trg) != 0):
		return Levenshtein.ratio(str_src, str_trg)
	else:
		return 0.0	

def get_jaro_winkler_similarity(str_src,str_trg):
	if  (len(str_src) != 0 and len(str_trg) != 0):
		return jfish.jaro_winkler_similarity(str_src, str_trg)
	else:
		return 0.0	

def get_col_value(row : any,col_list : list) -> str:
	val_list = []
	for col in col_list:
		val_list.append(str(row[col]).strip())
	return " ".join(val_list)

import src.settings.constants as const
def find_dup_row_by_sequence(df,sort_col):
	df = df.sort_values(by=[sort_col])
	list_dup_rows = []
	# Now Iterate through Each record and try to find out Group
	for src_idx in range(len(df)):
		src_row = df.iloc[src_idx]
		for trg_idx in range(len(df)):
			trg_row = df.iloc[trg_idx]
			#logger.info(f"source index {src_idx} ,target index : {trg_idx}")
			if trg_idx > src_idx:
				dup_grp = {}
				dup_grp[const.COL_SOURCE_ID] = src_row[const.COL_ID]
				dup_grp[const.COL_TARGET_ID] = trg_row[const.COL_ID]
				dup_grp["src_Company Name"] = str(src_row["Clean_Company"])
				dup_grp["trg_Company Name"] = str(trg_row["Clean_Company"])
				id, rsp = compare_rows(src_row,trg_row)
				#logger.info(id, rsp)
				dup_grp[const.COL_FUZZ_SIMILARITY] = float(rsp["f" + str(id)])
				dup_grp[const.COL_LEVENSHTEIN_SIMILARITY] = float(rsp["l" + str(id)])
				dup_grp[const.COL_JARO_SIMILARITY] = float(rsp["j" + str(id)])
				dup_grp[const.COL_ERROR_RATE] = float(rsp["e" + str(id)])
				if dup_grp[const.COL_ERROR_RATE] <= const.MAX_ERROR_RATE:
					list_dup_rows.append(dup_grp)
	return list_dup_rows

In [35]:
def find_dup_row_by_company_website(df,col_first_name="Clean_Company"):
	list_dup_rows = []
	df_dup = df.groupby(["FL_Comp_name"], as_index=False).agg(count=("FL_Comp_name", 'count'))
	df_dup = df_dup[df_dup["count"] > 1]
	for id, row in df_dup.iterrows():
		df_flt = df[df["FL_Comp_name"] == row["FL_Comp_name"]]
		df_flt = df_flt.sort_values(by=[col_first_name])
		list_dup_rows.extend(find_dup_row_by_sequence(df=df_flt,sort_col=col_first_name))
	
	# # Now Iterate through dup Rows and If last name matching Error rate is higher than threshold,
	# # Remove those rows from List.
	# list_dup_rows = remove_dup_rows_by_lastname_match_similarity(list_dup_rows)
	
	df_dup_rows = pd.DataFrame.from_dict(list_dup_rows)
	return df_dup_rows

In [36]:
def clean_company_name(company_name):
    # Define keywords to be removed
    keywords = ['LLP', 'LLC', 'INC', 'PVT', 'LTD', 'CORP', 'CO']
    # Create a regex pattern to match any of the keywords
    pattern = r'\b(?:' + '|'.join(keywords) + r')\b\.?'
    # Substitute the keywords with an empty string
    cleaned_name = re.sub(pattern, '', company_name, flags=re.IGNORECASE)
    # Remove extra spaces
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()
    return cleaned_name

def remove_https(url):
    # Check if the URL starts with "https://"
    cleaned_url = re.sub(r'^(http://|https://)?(www\.)?', '', url)
    return cleaned_url

In [37]:
import os
import time
from src.util import helper
import re
from src.settings import constants as const
from src.util import helper 
import pandas as pd

from src.webscrapper.webscrapper import get_company_website

def fill_company_website(df):
    for index, row in df.iterrows():
        if not row['Website']:  # Check if the website is empty
            rsp = get_company_website(row['Company'])
			
            if (get_fuzzy_similarity(row['Company'].strip().lower(),rsp["Link"].strip().lower())*100) >=20:
                df.at[index, 'Website'] = rsp['Link']
                df.at[index, 'Company'] = rsp['Company_name']
            else:
                df.at[index, 'Website'] = "Not Found"           
    return df
def read_file_prep_dataframe(file_path, file_ext):
	if file_ext.lower() == ".csv":
		df = pd.read_csv(file_path,encoding='cp1252')
	else:
		df = pd.read_excel(file_path) 

	df = df.fillna("") # Fill Empty value as all columns are strings only
		#df.to_excel("./output/crm_contact_with_row_no.xlsx")

	# Clean text of all dataframe text data one time
	df = df.map(lambda x: helper.clean_contact_data(x) if isinstance(x, str) else x)
	print(df.head())
	df = fill_company_website(df)
	df['Clean_Company']= df["Company"].apply(clean_company_name)
	df["Clean_Website"] = df["Website"].apply(remove_https)

	## Add New Columns to Add into original Dataframe
	df.insert(0,const.COL_DUP_GROUP_ID,999999)
	if const.COL_ID not in df.columns:
		# Add RowNum to DataFrame
		df[const.COL_ID] = range(0+1,len(df)+1)
	df.insert(2,const.COL_ERROR_RATE,100.00)
	df[const.COL_FUZZ_SIMILARITY] = 0.0
	df[const.COL_LEVENSHTEIN_SIMILARITY] = 0.0
	df[const.COL_JARO_SIMILARITY] = 0.0
	df[const.COL_DUP_ROW_GROUP] = ""
	
	df['FL_Comp_name'] = df['Company'].apply(lambda x: x[0] if len(x) > 0 else '')
	return df

def save_file(df: any, file_path : str, ext:str):
	if ext.lower() == ".xlsx":
		df.to_excel(file_path,index=False)
	else:
		df.to_csv(file_path,index=False)

def load_file(file_path : str, ext:str):
	if ext.lower() == ".csv":
		df = pd.read_csv(file_path,encoding='cp1252')
	else:
		df = pd.read_excel(file_path) 
	return df

def process_contact_duplicator(file_path : str):
	start_time = time.time()	
	try:
		if not os.path.exists(file_path):
			return
		name, ext = helper.get_file_name_and_extension(file_path)
		if not (ext.lower() == ".csv" or ext.lower() == ".xlsx"):
			return 
		# Step 1
		file_out = const.OUTPUT_PATH + "/" +  name + "_org"+ext
		if not os.path.exists(file_out):
			df = read_file_prep_dataframe(file_path,ext)
			save_file(df,file_out,ext)
		else:
			df = load_file(file_out,ext)
			#return df
		
		df["Domain"] = df["Clean_Website"].apply(lambda x: x.split('.')[0])
		
		file_out = const.OUTPUT_PATH + "/" +  name + "_dup"+ext
		if not os.path.exists(file_out):
			#Step 2
			print(df.head())
			df_dup = find_dup_row_by_company_website(df)
			#Step 3
	
			df = assign_dup_row_groups(df,df_dup)
			df.to_csv("Output_1.csv",date_format=str)
			df.to_excel("Output_1.xlsx",index=False)
			save_file(df,file_out,ext)
			return df
	
		# #Step 4 Assing the Keep / Duplicate Flag to each row based on research.
		# df = assign_keep_duplicate_flag(df)

		# file_out = const.OUTPUT_PATH + "/" +  name + "_with_flag"+ext
		# logger.info(f"Saving file with duplicate groups and Keep delete flaf {file_out}")
		# save_file(df,file_out,ext)
	except Exception as e:
		print(e)


In [38]:
def find_group(row, row_to_group, group_id):
	if row in row_to_group:
			return row_to_group[row]
	row_to_group[row] = group_id
	return group_id
def union_groups(row1, row2, row_to_group, group_id):
	group1 = find_group(row1, row_to_group, group_id)
	group2 = find_group(row2, row_to_group, group_id)
	if group1 != group2:
			for row in row_to_group:
					if row_to_group[row] == group2:
							row_to_group[row] = group1

def assign_group_ids(pairs):
	row_to_group = {}
	group_id = 1

	for src_row, trg_row in pairs:
			if src_row not in row_to_group and trg_row not in row_to_group:
					row_to_group[src_row] = group_id
					row_to_group[trg_row] = group_id
					group_id += 1
			elif src_row in row_to_group and trg_row not in row_to_group:
					row_to_group[trg_row] = row_to_group[src_row]
			elif trg_row in row_to_group and src_row not in row_to_group:
					row_to_group[src_row] = row_to_group[trg_row]
			else:
					union_groups(src_row, trg_row, row_to_group, group_id)

	# Ensure each component has a unique group ID
	unique_groups = {}
	current_group_id = 1
	for row in row_to_group:
			old_group_id = row_to_group[row]
			if old_group_id not in unique_groups:
					unique_groups[old_group_id] = current_group_id
					current_group_id += 1
			row_to_group[row] = unique_groups[old_group_id]

	return row_to_group

def assign_dup_row_groups(df,dup_df):
	dup_df = dup_df.sort_values(by=["src_row_no"])		

	# This will give Group ID assign to each row	
	group_ids = assign_group_ids(list(dup_df[['src_row_no', 'trg_row_no']].itertuples(index=False,name=None)))
	
	# In this Loop If any two record qualify or three records, One Record data is missing.
	for src_idx in range(len(dup_df)):
		row = dup_df.iloc[src_idx]
		
		if df[df[const.COL_ID] == row[const.COL_SOURCE_ID]][const.COL_DUP_GROUP_ID].values[0] == 999999:
			df.loc[df[const.COL_ID] == row[const.COL_SOURCE_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],group_ids[row[const.COL_SOURCE_ID]],str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]
		else:
			df.loc[df[const.COL_ID] == row[const.COL_TARGET_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],group_ids[row[const.COL_TARGET_ID]],str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]
	
	#Loop Throuh again with Dup Record for any Source Or Target Row is missing For Group ID Update
	for src_idx in range(len(dup_df)):
		row = dup_df.iloc[src_idx]
		grp_id = df[df[const.COL_ID] == row[const.COL_SOURCE_ID]][const.COL_DUP_GROUP_ID].values[0]
		if grp_id != 999999:
			df.loc[df[const.COL_ID] == row[const.COL_TARGET_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],grp_id,str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]
		else:
			grp_id = df[df[const.COL_ID] == row[const.COL_TARGET_ID]][const.COL_DUP_GROUP_ID].values[0]
			df.loc[df[const.COL_ID] == row[const.COL_SOURCE_ID], [const.COL_FUZZ_SIMILARITY, const.COL_LEVENSHTEIN_SIMILARITY, \
					const.COL_JARO_SIMILARITY,const.COL_ERROR_RATE,const.COL_DUP_GROUP_ID,const.COL_DUP_ROW_GROUP]] \
					= [row[const.COL_FUZZ_SIMILARITY],row[const.COL_LEVENSHTEIN_SIMILARITY],row[const.COL_JARO_SIMILARITY], \
					row[const.COL_ERROR_RATE],grp_id,str(row[const.COL_SOURCE_ID])+"-"+str(row[const.COL_TARGET_ID])]

	return df

In [ ]:
process_contact_duplicator("./data/Compant_Data_1.2.csv")

In [49]:
df = pd.read_csv("./output/Company_Data_org.csv")

In [ ]:
list_dup_rows = []
df_dup = df.groupby(["fname_Company"], as_index=False).agg(count=("fname_Company", 'count'))
df_dup = df_dup[df_dup["count"] > 1]
for id, row in df_dup.iterrows():
	df_flt = df[df["fname_Company"] == row["fname_Company"]]
	df_flt = df_flt.sort_values(by=["Company"])
	df_dup_lname = df_flt.groupby(["Company"], as_index=False).agg(count=("Company", 'count'))
	df_dup_lname = df_dup_lname[df_dup_lname["count"] > 1]
	for id1, row in df_dup_lname.iterrows():
		df_flt_lname = df_flt[df_flt['Company'] == row['Company']]
		df_flt_lname = df_flt_lname.sort_values(by=["Company"])
		

In [ ]:
df_flt_lname

In [1]:
import pandas as pd
df = pd.read_csv("./data/Compant_Data_1.2.csv")

In [ ]:
df.head()

In [ ]:
from src.duplicate_finder.Company_Duplicate_finder import process_company_duplicator
df = process_company_duplicator("./data/Compant_Data_1.2.csv")

In [3]:
df = df.fillna("")

In [4]:
from src.webscrapper.webscrapper import get_company_website

def fill_company_website(df):
    for index, row in df.iterrows():
        if not row['Website']:  # Check if the website is empty
            rsp = get_company_website(row['Company'])
            if rsp['Link'] !="Not Found":
                df.at[index, 'Website'] = rsp['Link']
                df.at[index, 'Company'] = rsp['Company_name']
            else:
                df.at[index, 'Website'] = "Not Found"
          
               
    return df
    

In [ ]:
df = fill_company_website(df)

In [ ]:
df.head()

In [ ]:
from src.webscrapper.webscrapper import get_company_website
get_company_website("BDO")

In [4]:
from src.webscrapper.webscrapper import get_google_api_resposne_search_items
rsp = get_google_api_resposne_search_items("BDO +website+")

In [ ]:
get_company_website("beyond markets")

In [ ]:
rsp['data'][0].get('snippet') is not None

In [ ]:
get_company_website("ab bernstein")

In [ ]:
get_google_api_resposne_search_items("www.bernsteinresearch.com")

In [ ]:
get_google_api_resposne_search_items("www.akemona.com")

In [ ]:
get_google_api_resposne_search_items("www.withbanner.com")

In [ ]:
get_company_website("Banner Technologies")

In [ ]:
# Company Name and title will Different 
get_google_api_resposne_search_items("Banner Technologies Website")

In [ ]:
# website Is Not Working still it will give me the result
get_google_api_resposne_search_items("http://caplab.ai")

In [ ]:
#Wrong Website
get_company_website("CapLab")

In [5]:
import re
def clean_company_name(company_name):
    # Define keywords to be removed
    keywords = ['LLP', 'LLC', 'INC', 'PVT', 'LTD', 'CORP', 'CO']
    # Create a regex pattern to match any of the keywords
    pattern = r'\b(?:' + '|'.join(keywords) + r')\b\.?'
    # Substitute the keywords with an empty string
    cleaned_name = re.sub(pattern, '', company_name, flags=re.IGNORECASE)
    # Remove extra spaces
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()
    return cleaned_name

In [ ]:
def remove_https(url):
    # Check if the URL starts with "https://"
    if url.startswith("https://"):
        # Remove "https://" from the URL
        return url[8:]
    return url

In [6]:
df['Company']= df['Company'].apply(clean_company_name)

In [13]:
import pandas as pd
df = pd.read_excel("Output_1.xlsx")

In [ ]:
df.head()

In [15]:
df["action"] = ""
df["Real_Name"] =""


In [16]:
def assign_exact_duplicate_flag(df : any):
	df_flt = df[df['dup_group_id'] != 999999]
	df_grp = df_flt.groupby(["dup_group_id","Clean_Website"], as_index=False).agg(count=("dup_group_id", 'count'))
	list_exact_dup = df_grp[df_grp["count"] > 1]["dup_group_id"].unique().tolist()
	for exact_dup_id in list_exact_dup:
		df_tmp = df[df["dup_group_id"] == exact_dup_id]
		df_tmp = df_tmp.sort_values(by=["Company"], ascending=False)		
		for idx in range(len(df_tmp)):
			row = df_tmp.iloc[idx]
			if idx == 0:
				flag = "Keep"
			else:
				flag = "Duplicate"	
			df.loc[(df["dup_group_id"] == exact_dup_id) & (df["RowNo"] == row["RowNo"]), ["action"]] = [flag]
	return df, list_exact_dup

In [17]:
df,list_exact_dup = assign_exact_duplicate_flag(df)

In [ ]:
df.head()

In [3]:
df_flt = df[df['dup_group_id'] != 999999]
df_grp = df_flt.groupby(["dup_group_id","Company","Website"],as_index=False).agg(count=("dup_group_id", 'count'))

In [ ]:
df_grp["Website"]

In [ ]:
from src.webscrapper.webscrapper import get_website_robot_file
robot_file = get_website_robot_file( "www.abacasexchange.com","abacaschange")
robot_file

In [ ]:
from src.webscrapper.webscrapper import get_google_api_resposne_search_items
get_google_api_resposne_search_items("www.abacaange.com")

In [35]:
import src.settings.constants as const
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import src.webscrapper.webscrapper as wsc
def search_contact_information(Website,Company):
    rsp = {}
    rsp["msg"] = ""
    rsp["Company_name"] = const.NOT_FOUND
    robot_file = wsc.get_website_robot_file( Website,Company)
    page_rsp = wsc.get_page_content(Website,robot_file)
    if page_rsp["is_success"]:
        phrases = re.findall(r'\b[A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)*\b', page_rsp["page_text"])
        similar_phrases = process.extract(Company, phrases, scorer=fuzz.token_sort_ratio, limit=1)
        rsp["Company_name"]= similar_phrases[0][0]
    
    return rsp




In [18]:
def validate_url(url):
    if not url.startswith('https://'):
        # If it doesn't, add 'https://'
        url = 'https://' + url
    if not url.startswith("https://www."):
        url = "https://www." + url
    return url

In [19]:
import src.webscrapper.webscrapper as wsc
import src.settings.constants as const
import re
def search_Company_information(Website, Company):
    rsp = {}
    rsp["msg"] = ""
    rsp["Company_name"] = const.NOT_FOUND
    website_valid = validate_url(Website)
    robot_file = wsc.get_website_robot_file( website_valid, Company)
    page_rsp = wsc.get_page_content(website_valid, robot_file)
    
    if page_rsp["is_success"]:
        page_text = str(page_rsp["page_text"])
        str_input=re.sub(r'[^\x20-\x7E]', '', page_text)
        str_input=re.sub(r'\s+', ' ', str_input)
        str_input = str_input.strip().lower()  # Ensure proper indentation here
        if str_input.find(Company.lower()) > -1:  # Comparing in lowercase for consistency
            rsp["Company_name"] = Company
    return rsp


In [ ]:
similar_phrases[0][0]

In [9]:
def validate_url(url):
    if not url.startswith('https://'):
        # If it doesn't, add 'https://'
        url = 'https://' + url
    if not url.startswith("https://www."):
        url = "https://www." + url
    return url


In [20]:
def assign_probable_duplicate_flag(df : any, list_exact_dup : list[int]):
	# Remove Exact Duplicate rows from Dataframe
	df_flt = df[df['dup_group_id'] != 999999]
	for exact_dup_id in list_exact_dup:
		df_flt = df_flt[df_flt['dup_group_id'] != exact_dup_id]
	# Sort by Dup Group ID
	df_flt = df_flt.sort_values(by=["dup_group_id"])
	for id, row in df_flt.iterrows():
		rsp=search_Company_information(row['Website'],row["Company"])
		if rsp['Company_name'] != "Not found":
			df.at[id, 'Real_Name'] = rsp['Company_name']
			df.at[id,"action"] = "Keep"
		else:
			df.at[id, 'action'] ="Not Sure"
	
	return df

In [ ]:
df = assign_probable_duplicate_flag(df,list_exact_dup)

In [22]:
df.to_excel("Flag_Assigned_Duplicat_records_2.xlsx",index=False)

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

text = """
Apple Inc. is a technology company known for its iPhone and Mac products. 
Microsoft Corporation develops Windows and Office software. 
Amazon is an e-commerce giant. 
Google, under Alphabet Inc., dominates the search engine market.
"""

phrases = re.findall(r'\b[A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)*\b', text)
print(phrases)

# Provided company name
company_name = "Apple Inc."


similar_phrases = process.extract(company_name, phrases, scorer=fuzz.token_sort_ratio, limit=1)


print(f"Similar phrases to '{company_name}':")
for phrase, similarity in similar_phrases:
    print(f"{phrase}: {similarity}%")


In [ ]:
phrases

In [ ]:
import re
text = """
Apple Inc. is a technology company known for its iPhone and Mac products. 
Microsoft Corporation develops Windows and Office software. 
Amazon is an e-commerce giant. 
Google, under Alphabet Inc., dominates the search engine market.
"""
re.findall(text,"Microsoft Corporation")

In [8]:
pattern = r'\b' + re.escape("Apple INC.") + r'\b'

In [9]:
matches = re.findall(pattern, text)

In [ ]:
matches

In [ ]:
import re

def find_apple(text):
    # Define the company name to search for
    company = "Apple Inc."
    
    # Use word boundaries (\b) to ensure exact matches
    pattern = r'\b' + re.escape(company) + r'\b'
    
    # Find matches using re.findall
    matches = re.findall(pattern, text)
    
    # Return the company name if it is found, otherwise return None
    return company if matches else None

# Example usage
page_text = """Apple Inc. is a technology company known for its iPhone and Mac products. 
Microsoft Corporation develops Windows and Office software. 
Amazon is an e-commerce giant. 
Google, under Alphabet Inc., dominates the search engine market."""

matched_company = find_apple(page_text)
print(matched_company)


In [1]:
page_text = """Apple Inc. is a technology company known for its iPhone and Mac products. 
Microsoft Corporation develops Windows and Office software. 
Amazon is an e-commerce giant. 
Google, under Alphabet Inc., dominates the search engine market."""

In [ ]:
page_text.find("Ap")

In [2]:
import src.webscrapper.webscrapper as wsc

In [ ]:
wsc.get_page_content("www.acryliccap.com","not define")

In [ ]:
wsc.get_page_content("https://www.aerocapitalsolutions.com","")

In [ ]:
wsc.get_page_content("www.beyondcapitalmarkets.com","")

In [ ]:
wsc.get_page_content("https://www.bluetradingsystems.com","")

In [ ]:
wsc.get_page_content("www.capconcorp.com","")

In [ ]:
wsc.get_page_content("http://caplab.ai","")

In [3]:

values = wsc.get_google_api_resposne_search_items("bids trading")

In [ ]:
type(values)

In [7]:
def validate_url(url):
    if not url.startswith('https://'):
        # If it doesn't, add 'https://'
        url = 'https://' + url
    if not url.startswith("https://www."):
        url = "https://www." + url
    return url


In [ ]:
validate_url("www.actualgroup.io")

In [ ]:
import src.duplicate_finder.Company_Duplicate_finder as cdup
df = cdup.process_company_duplicator("./data/Compant_Data_1.2.csv")

In [2]:
df.to_excel("Company_Data_1.2_with_flag_1.xlsx",index=False)

In [1]:
import src.webscrapper.webscrapper as wsc
rsp = wsc.get_google_api_resposne_search_items("legal name of company 'BTIG'")

In [7]:
import re

def extract_legal_name(text):
    pattern = r'\b[A-Za-z0-9\s,]+?\s(Inc\.|LLC|Ltd\.|Corp\.|Corporation|Co\.|Limited|LLP|L\.P\.|PLC|GmbH|S\.A\.|Pty\. Ltd\.|BV)\b'
    match = re.search(pattern, text)
    if match:
        return match.group(0)
    else:
        return "Not found"

for item in rsp["data"]:
	input_text = item["title"] + " " +  item["snippet"]
	cname = extract_legal_name(input_text)
	if cname != "Not found":
		print(cname)
		print(item["link"])
		print(input_text)


BTIG, LLC
https://www.sec.gov/Archives/edgar/data/1178937/000117893720000007/rePublic.pdf
BTIG, LLC Dec 31, 2019 ... ADDRESS OF PRINCIPAL PLACE OF BUSINESS: (Do note use P.O. Box No.) 600 Montgomery Street 6 Floor. OFFICIAL USE ONLY. FIRM I.D. NO. San Francisco.
Bartlett LLP
https://www.btig.com/news/btig-adds-winston-kitchingham-to-legal-team/
BTIG Adds Winston Kitchingham to Legal Team - BTIG May 23, 2022 ... Kitchingham began his career as a corporate lawyer at Simpson Thacher & Bartlett LLP and Dewey Ballantine LLP. BTIG Investment Banking and ...
BTIG, LLC
https://files.brokercheck.finra.org/firm/firm_122225.pdf
BrokerCheck Report - BTIG, LLC Apr 28, 2017 ... This firm was formed in Delaware on 06/05/2002. CRD#. This section provides the brokerage firm's full legal name, "Doing Business As" name, ...
BTIG, LLC
https://brokercheck.finra.org/firm/summary/122225
BTIG, LLC - BrokerCheck Brokerage Firm Regulated by FINRA (San Francisco district office) MAIN ADDRESS 350 BUSH STREET 9TH 